To begin, The following code generates a network with one fully-connected hidden layer with 256 units and ReLU activation, followed by a fully-connected output layer with 10 units (no activation function).

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.21109891,  0.0387467 , -0.14684327,  0.12837288,  0.22814858,
        -0.28069234,  0.08947153,  0.14173885,  0.14360254,  0.29423195],
       [ 0.13008785,  0.00457772, -0.00087339,  0.26544315,  0.11481602,
        -0.1414393 , -0.02059632, -0.00844908,  0.24608138,  0.02118239]],
      dtype=float32)>

In this example, we constructed our model by instantiating an keras.models.Sequential, with layers in the order that they should be executed passed as arguments. In short, Sequential defines a special kind of keras.Model, the class that presents a block in Keras. It maintains an ordered list of constituent Models. Note that each of the two fully-connected layers is an instance of the Dense class which is itself a subclass of Model. The forward propagation (call) function is also remarkably simple: it chains each block in the list together, passing the output of each as the input to the next. Note that until now, we have been invoking our models via the construction net(X) to obtain their outputs

# A Custom Block
Perhaps the easiest way to develop intuition about how a block works is to implement one ourselves. Before we implement our own custom block, we briefly summarize the basic functionality that each block must provide:
   - Ingest input data as arguments to its forward propagation function.
   - Generate an output by having the forward propagation function return a value. Note that the output may have a different shape from the input. For example, the first fully-connected layer in our model above ingests an input of arbitrary dimension but returns an output of dimension 256.
    - Calculate the gradient of its output with respect to its input, which can be accessed via its backpropagation function. Typically this happens automatically.
    - Store and provide access to those parameters necessary to execute the forward propagation computation.
    - Initialize model parameters as needed.

In the following snippet, we code up a block from scratch corresponding to an MLP with one hidden layer with 256 hidden units, and a 10-dimensional output layer. Note that the MLP class below inherits the class that represents a block

In [4]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden Layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10) # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X
    def call(self, X):
        return self.out(self.hidden((X)))

We instantiate the MLP’s layers in the constructor and subsequently invoke these layers on each call to the forward propagation function. Note a few key details. First, our customized __init__ function invokes the parent class’s __init__ function via super().__init__() sparing us the pain of restating boilerplate code applicable to most blocks. We then instantiate our two fully-connected layers, assigning them to self.hidden and self.out. Note that unless we implement a new operator, we need not worry about the backpropagation function or parameter initialization. The system will generate these functions automatically.

In [5]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.07101482, -0.2110543 , -0.16010809,  0.1866937 , -0.08635128,
         0.04839141, -0.08112621,  0.1518958 , -0.11969745,  0.43326932],
       [ 0.3469178 , -0.34356403,  0.05619559,  0.240516  ,  0.1905013 ,
        -0.02828831, -0.0613918 , -0.23613352,  0.07071781,  0.09410749]],
      dtype=float32)>

# The Sequential Block
We can now take a closer look at how the Sequential class works. Recall that Sequential was designed to daisy-chain other blocks together. To build our own simplified MySequential, we just need to define two key function: 1. A function to append blocks one by one to a list. 2. A forward propagation function to pass an input through the chain of blocks, in the same order as they were appended.

In [6]:
# The following MySequential class delivers the same functionality of the default Sequential class.
# A custom Sequential class
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, 'block' it an instance of a 'tf.keras.Layers.Layer'
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [7]:
""" When our MySequential’s forward propagation function is invoked, each added block is executed in the order in which they were added. 
We can now reimplement an MLP using our MySequential class. """
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10)
)
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.04223305,  0.25023624,  0.05408087,  0.14699587,  0.10596074,
         0.11279768,  0.30652404,  0.1820293 , -0.03227028,  0.07613305],
       [ 0.00723713,  0.18230537,  0.24777259,  0.10203905,  0.11089441,
         0.08180499,  0.18224534,  0.21758461,  0.04705925,  0.11204869]],
      dtype=float32)>

# Executing Core in Forward Propagation Function
The Sequential class makes model construction easy, allowing us to assemble new architectures without having to define our own class. However, not all architectures are simple daisy chains. When greater flexibility is required, we will want to define our own blocks. For example, we might want to execute Python’s control flow within the forward propagation function. Moreover, we might want to perform arbitrary mathematical operations, not simply relying on predefined neural network layers. You might have noticed that until now, all of the operations in our networks have acted upon our network’s activations and its parameters. Sometimes, however, we might want to incorporate terms that are neither the result of previous layers nor updatable parameters. We call these constant parameters.

In [8]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with 'tf.constant' are not updated
        # during training (constant parametes)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the relu and matmul functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In this FixedHiddenMLP model, we implement a hidden layer whose weights (self.rand_weight) are initialized randomly at instantiation and are thereafter constant. This weight is not a model parameter and thus it is never updated by backpropagation. The network then passes the output of this “fixed” layer through a fully-connected layer.

Note that before returning the output, our model did something unusual. We ran a while-loop, testing on the condition its L1 norm is larger than 1, and dividing our output vector by 2 until it satisfied the condition. Finally, we returned the sum of the entries in X. To our knowledge, no standard neural network performs this operation. Note that this particular operation may not be useful in any real-world task. Our point is only to show you how to integrate arbitrary code into the flow of your neural network computations.

In [9]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7086938>

In [11]:
# We can mix and match various ways of assembling blocks together. In the following example, we nest blocks in some creative ways
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6869493>

# Efficiency
The avid reader might start to worry about the efficiency of some of these operations. After all, we have lots of dictionary lookups, code execution, and lots of other Pythonic things taking place in what is supposed to be a high-performance deep learning library. The problems of Python’s global interpreter lock are well known. In the context of deep learning, we may worry that our extremely fast GPU(s) might have to wait until a puny CPU runs Python code before it gets another job to run. The best way to speed up Python is by avoiding it altogether

# Summary
    - Layers are blocks.
    - Many layers can comprise a block.
    - Many blocks can comprise a block.
    - A block can contain code.
    - Blocks take care of lots of housekeeping, including parameter initialization and backpropagation.
    - Sequential concatenations of layers and blocks are handled by the Sequential block.